![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F04+-+scikit-learn&dt=04b+-+Vertex+AI+Custom+Model+-+scikit-learn+-+Custom+Job+With+Python+Source+Distribution.ipynb)

# 04a - Vertex AI Custom Model - scikit-learn - Custom Job With Python File

**04 Series Overview**

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%scikit-learn%20-%20in%20Notebook.ipynb) notebook, the model training happened directly in the notebook.  The model was then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `04a-04i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI Training as managed jobs.  These jobs will be demonstrated as:

-  Custom Job that trains and saves (to GCS) a model from a python script (`04a`), python source distribution (`04b`), and custom container (`04c`)
-  Training Pipeline that trains and registers a model from a python script (`04d`), python source distribution (`04e`), and custom container (`04f`)
-  Hyperparameter Tuning Jobs from a python script (`04g`), python source distribution (`04h`), and custom container (`04i`)

**This Notebook (`04b`): An extension of `04a` that saves the script within a source distribution**

This notebook trains the same logistic regression model from [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%scikit-learn%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a Python script as shown in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%scikit-learn%20-%20Notebook%20to%20Script.ipynb).  

A Python source distribution is built containing the script.  For more guidance on this process visit the tip notebook [Python Packages](../Tips/Python%20Packages.ipynb).  The source distribution is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this notebook gives an example of making the shift.

This job is launched using the Vertex AI client library:
- [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference)
    - [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)
        - [`aiplatform` package](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
            - [`aiplatform.CustomJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob)
                - with [worker_pool_specs](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.WorkerPoolSpec) using `python_package_spec`

**Vertex AI Training**

In Vertex AI Training you can run your training code as a job where you specify the compute resources to use. For tips on preparing code, running training jobs, and workflows for building custom containers with software and training code combined please visit these [tips notebooks](../Tips/readme.md) in this repository:
- [Python Packages](../Tips/Python%20Packages.ipynb)
- [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb)
- [Python Training](../Tips/Python%20Training.ipynb)

<p align="center" width="100%">
    <img src="../architectures/overview/training.png" width="45%">
    &nbsp; &nbsp; &nbsp; &nbsp;
    <img src="../architectures/overview/training2.png" width="45%">
</p>

**Prerequisites:**
-  [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)
-  Understanding:
    -  Model overview in [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%scikit-learn%20-%20in%20Notebook.ipynb)
    -  Convert notebook code to Python Script in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%scikit-learn%20-%20Notebook%20to%20Script.ipynb)

**Resources:**
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [sklearn](https://scikit-learn.org/stable/)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
    - Pre-built Containers for Vertex AI
        - [Training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)
        - [Prediction & Explaination](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05b_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05b_console.png" width="45%">
</p>

---
## Setup

### Package Installs (if needed)

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [1]:
try:
    import build
except ImportError:
    print('You need to pip install build')
    !pip install build -q

### Environment

inputs:

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [3]:
REGION = 'us-central1'
EXPERIMENT = '04b'
SERIES = '04'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id-splits' + '-' + VAR_TARGET # add more variables to the string with space delimiters
SOLVER = 'newton-cg'
PENALTY = 'l2'

packages:

In [4]:
from google.cloud import aiplatform
from datetime import datetime
import pkg_resources
from IPython.display import Markdown as md
from google.cloud import bigquery
from google.cloud import storage
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import os, shutil
import numpy as np
import pandas as pd

clients:

In [5]:
aiplatform.init(project=PROJECT_ID, location=REGION)
gcs = storage.Client()
bq = bigquery.Client()

parameters:

In [6]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"
BLOB = f"{SERIES}/{EXPERIMENT}/models/{TIMESTAMP}/model/model.pkl"

In [7]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'mg-ce-demos-main@mg-ce-demos.iam.gserviceaccount.com'

List the service accounts current roles:

In [8]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/aiplatform.admin
roles/bigquery.admin
roles/editor
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [9]:
!rm -rf {DIR}
!mkdir -p {DIR}

Experiment Tracking:

In [10]:
FRAMEWORK = 'sklearn'
TASK = 'classification'
MODEL_TYPE = 'logistric-regression'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [11]:
tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})

In [12]:
tb.resource_name

'projects/633472233130/locations/us-central1/tensorboards/4577495604850065408'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [13]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

---
## Training

### Python File for Training

This notebook trains the same Tensorflow Keras model from [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a python script as shown in [05 - Vertex AI Custom Model - TensorFlow - Notebook to Script](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Notebook%20to%20Script.ipynb) which stores the script in [`./code/train.py`](./code/train.py).

**Review the script:**

In [14]:
SCRIPT_PATH = './code/train.py'

with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python


# package import
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

import pickle
import pandas as pd 
import numpy as np 

from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud import storage
import argparse
import os
import sys

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--penalty', dest = 'penalty', default = 'l2', type = str, help = 'Penalty term')
parser.add_argument('--solver', dest = 'solver', default = 'newton-cg', type = str, help = 'Logistic regression solver')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str)
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--bq_project', dest = 'bq_project', type=str)
parser.add_argument('--bq_dataset', dest = 'bq_dataset', type=str)
parser.add_argument('--bq_table', dest = 'bq_table', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest = 'experiment', type=str)
parser.add_argument('--series', dest = 'series', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)
parser.add_argument('--bucket', dest = 'bucket', type=str)
parser.add_argument('--uri', dest = 'uri', type=str)
parser.add_argument('--blob', dest = 'blob', type=str)
parser.add_argument('--timestamp', dest = 'timestamp', type=str)
args = parser.parse_args()

# Model Training
VAR_TARGET = str(args.var_target)
VAR_OMIT = str(args.var_omit).split('-')

# clients
bq = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)

# Vertex AI Experiment
expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment': args.experiment, 'series': args.series, 'project_id': args.project_id})

# get schema from bigquery source
query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{args.bq_table}'"
schema = bq.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bq.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE {VAR_TARGET} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}', 'nclasses': nclasses, 'var_split': 'splits', 'var_target': VAR_TARGET})

train_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'TRAIN'"
train = bq.query(train_query).to_dataframe()
X_train = train.loc[:, ~train.columns.isin(VAR_OMIT)]
y_train = train[VAR_TARGET].astype('int')

val_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'VALIDATE'"
val = bq.query(val_query).to_dataframe()
X_val = val.loc[:, ~val.columns.isin(VAR_OMIT)]
y_val = val[VAR_TARGET].astype('int')

test_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'TEST'"
test = bq.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(VAR_OMIT)]
y_test = test[VAR_TARGET].astype('int')

# Logistic Regression
# instantiate the model 
logistic = LogisticRegression(solver=args.solver, penalty=args.penalty)

# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

expRun.log_params({'solver': args.solver, 'penalty': args.penalty})

# define pipeline
pipe = Pipeline(steps=[("scaler", scaler), ("logistic", logistic)])

# define grid search model
model = pipe.fit(X_train, y_train)

# test evaluations:
y_pred = model.predict(X_test)
test_acc = metrics.accuracy_score(y_test, y_pred) 
test_prec = metrics.precision_score(y_test, y_pred)
test_rec = metrics.recall_score(y_test, y_pred)
test_rocauc = metrics.roc_auc_score(y_test, y_pred)
expRun.log_metrics({'test_accuracy': test_acc, 'test_precision': test_prec, 'test_recall': test_rec, 'test_roc_auc': test_rocauc})

# val evaluations:
y_pred_val = model.predict(X_val)
val_acc = metrics.accuracy_score(y_val, y_pred_val) 
val_prec = metrics.precision_score(y_val, y_pred_val)
val_rec = metrics.recall_score(y_val, y_pred_val)
val_rocauc = metrics.roc_auc_score(y_val, y_pred_val)
expRun.log_metrics({'validation_accuracy': val_acc, 'validation_precision': val_prec, 'validation_recall': val_rec, 'validation_roc_auc': val_rocauc})

# training evaluations:
y_pred_training = model.predict(X_train)
training_acc = metrics.accuracy_score(y_train, y_pred_training) 
training_prec = metrics.precision_score(y_train, y_pred_training)
training_rec = metrics.recall_score(y_train, y_pred_training)
training_rocauc = metrics.roc_auc_score(y_train, y_pred_training)
expRun.log_metrics({'training_accuracy': training_acc, 'training_precision':training_prec, 'training_recall': training_rec, 'training_roc_auc': training_rocauc})

# output the model save files
with open('model.pkl','wb') as f:
    pickle.dump(model,f)
    
# Upload the model to GCS
bucket = storage.Client().bucket(args.bucket)
blob = bucket.blob(args.blob)
blob.upload_from_filename('model.pkl')

expRun.log_params({'model.save': f'{args.uri}/models/{args.timestamp}/model'})
expRun.end_run()

```

### Construct Python Package

In [15]:
DIR

'temp/04b'

In [16]:
os.listdir(DIR)

[]

#### Create the folder structure

In [17]:
os.makedirs(DIR + f'/src/{EXPERIMENT}_trainer')

In [18]:
for root, dirs, files in os.walk(DIR):
    print(root)

temp/04b
temp/04b/src
temp/04b/src/04b_trainer


#### Add files to directory
Copy the `./code/train.py` file

In [19]:
shutil.copyfile(SCRIPT_PATH, f'{DIR}/src/{EXPERIMENT}_trainer/train.py')
with open(f'{DIR}/src/{EXPERIMENT}_trainer/__init__.py', 'w') as file: pass

create `setup.py` file:

In [20]:
%%writefile {DIR}/src/setup.py
from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['google-cloud-aiplatform', 'protobuf',  'db-dtypes>=1.0.0', 'google-auth>=2.6.0', 'google-cloud-bigquery>=3.0.1']

setup(
    name = '04b_trainer',
    version = '0.1',
    install_requires = REQUIRED_PACKAGES, 
    packages = find_packages(),
    include_package_data = True,
    description='Training Package'
)

Writing temp/04b/src/setup.py


add `__init__.py` file to the trainer modules folder:

In [21]:
!touch {DIR}/src/{EXPERIMENT}_trainer/__init__.py

list directory:

In [22]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/04b/src/setup.py
temp/04b/src/04b_trainer/__init__.py
temp/04b/src/04b_trainer/train.py


#### Build the Python distribution archives:

Create the source distribution and copy it to the projects storage bucket:
- change to the local direcory with the source folder
- remove any previous distributions
- tar and gzip the source folder
- copy the distribution to the project folder on GCS
- change back to the local project directory

In [23]:
%cd {DIR}

!rm -f src.tar src.tar.gz
!tar cvf src.tar src
!gzip src.tar

%cd ..
%cd ..

/Users/mikegoodman/Documents/developer/git_repos/vertex-ai-mlops/04 - scikit-learn/temp/04b
a src
a src/setup.py
a src/04b_trainer
a src/04b_trainer/__init__.py
a src/04b_trainer/train.py
/Users/mikegoodman/Documents/developer/git_repos/vertex-ai-mlops/04 - scikit-learn/temp
/Users/mikegoodman/Documents/developer/git_repos/vertex-ai-mlops/04 - scikit-learn


list directory:

In [24]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/04b/src.tar.gz
temp/04b/src/setup.py
temp/04b/src/04b_trainer/__init__.py
temp/04b/src/04b_trainer/train.py


#### Copy to GCS

Here the folder structure for DIR will be copied to the GCS Bucket used across this project.  This section uses skills that are discussed in more detail in the [Python Client for GCS](../Tips/Python%20Client%20for%20GCS.ipynb) notebook.

List buckets in project:

In [25]:
list(gcs.list_buckets())

[<Bucket: artifacts.mg-ce-demos.appspot.com>,
 <Bucket: cloud-ai-platform-585d7202-46a7-4364-961f-f1aed4f943a3>,
 <Bucket: dataflow-staging-us-central1-633472233130>,
 <Bucket: dataprep-staging-0b868edb-9e2e-49a0-9c95-1eb997f327be>,
 <Bucket: gcf-sources-633472233130-us-central1>,
 <Bucket: mg-ce-demos>,
 <Bucket: mg-ce-demos-aip-20220314182836>,
 <Bucket: mg-ce-demos-bucket>,
 <Bucket: mg-ce-demos-l300>,
 <Bucket: mg-ce-demos-vertex-staging-us-central1>,
 <Bucket: mg-ce-demos_cloudbuild>,
 <Bucket: mg-ce-demosaip-20220314191437>,
 <Bucket: us.artifacts.mg-ce-demos.appspot.com>]

Get the bucket:

In [26]:
bucket = gcs.lookup_bucket(PROJECT_ID)

In [27]:
bucket

<Bucket: mg-ce-demos>

copy files to GCS bucket:

In [28]:
!gsutil cp {DIR}/src.tar.gz {URI}/{TIMESTAMP}/src.tar.gz

Copying file://temp/04b/src.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      


In [29]:
print(f"View the bucket directly here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES}/{EXPERIMENT};tab=objects&project={PROJECT_ID}")

View the bucket directly here:
https://console.cloud.google.com/storage/browser/mg-ce-demos/04/04b;tab=objects&project=mg-ce-demos


list files in bucket:

In [30]:
for blob in list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/')):
    print(blob.name)

04/04b/20230125091901/src.tar.gz


get a list of source distributions and select the latest:

In [72]:
SOURCE_LIST = [f'gs://{PROJECT_ID}/{blob.name}' for blob in list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/')) if blob.name[-7:] == '.tar.gz']
print(SOURCE_LIST)
SOURCE = [SOURCE_LIST[-1]]
print(SOURCE)

['gs://mg-ce-demos/04/04b/20230125091901/src.tar.gz']
['gs://mg-ce-demos/04/04b/20230125091901/src.tar.gz']


### Setup Training Job

In [34]:
CMDARGS = [
    "--penalty=" + PENALTY,
    "--solver=" + SOLVER,
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME,
    "--bucket=" + BUCKET,
    "--uri=" + URI,
    "--blob=" + BLOB,
    "--timestamp=" + TIMESTAMP
]

In [35]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": SOURCE,
            "python_module": f"{EXPERIMENT}_trainer.train",
            "args": CMDARGS
        }
    }
]

In [36]:
customJob = aiplatform.CustomJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/models/{TIMESTAMP}",
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

### Run Training Job

In [37]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    #tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/633472233130/locations/us-central1/customJobs/5943752189498359808
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/633472233130/locations/us-central1/customJobs/5943752189498359808')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5943752189498359808?project=633472233130
CustomJob projects/633472233130/locations/us-central1/customJobs/5943752189498359808 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/5943752189498359808 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/5943752189498359808 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/5943752189498359808 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/633472233130/locations/us-central1/customJobs/5943752

In [38]:
customJob.display_name

'04_04b_20230125091901'

In [39]:
customJob.resource_name

'projects/633472233130/locations/us-central1/customJobs/5943752189498359808'

Create hyperlines to job and tensorboard here:

In [40]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
board_link = f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{customJob.resource_name.split('/')[-1]}"

print(f'Review the Custom Job here:\n{job_link}')
print(f'Review the TensorBoard From the Job here:\n{board_link}')

Review the Custom Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/5943752189498359808/cpu?cloudshell=false&project=mg-ce-demos
Review the TensorBoard From the Job here:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+633472233130+locations+us-central1+tensorboards+4577495604850065408+experiments+5943752189498359808


---
## Serving

### Upload The Model

In [41]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  modelmatch[0].resource_name,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )

Model Already in Registry:
Loading model as new default version.
Creating Model
Create Model backing LRO: projects/633472233130/locations/us-central1/models/model_04_04b/operations/6062642038212395008
Model created. Resource name: projects/633472233130/locations/us-central1/models/model_04_04b@3
To use this Model in another session:
model = aiplatform.Model('projects/633472233130/locations/us-central1/models/model_04_04b@3')


>**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`
>
>**Retrieve a Model Resource**
>[aiplatform.Model()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
>```Python
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

In [42]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_04_04b?project=mg-ce-demos


### Vertex AI Experiment Update and Review

In [43]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

In [44]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'customJobs.display_name': customJob.display_name,
    'customJobs.resource_name': customJob.resource_name,
    'customJobs.link': job_link,
    'customJobs.tensorboard': board_link
})

Complete the experiment run:

In [45]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Retrieve the experiment:

In [46]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [47]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.data_source,param.experiment,param.model.display_name,param.model.uri,param.nclasses,param.customJobs.tensorboard,...,metric.test_accuracy,metric.test_recall,metric.validation_recall,metric.training_precision,metric.test_precision,metric.validation_roc_auc,metric.training_accuracy,metric.training_roc_auc,metric.validation_precision,metric.training_recall
0,experiment-04-04b-sklearn-classification-logis...,run-20230125091901,system.ExperimentRun,COMPLETE,bq://mg-ce-demos.fraud.fraud_prepped,04b,04_04b,gs://mg-ce-demos/04/04b/models/20230125091901/...,2.0,https://us-central1.tensorboard.googleusercont...,...,0.999087,0.607143,0.54902,0.878229,0.894737,0.774422,0.99921,0.809018,0.848485,0.618182
1,experiment-04-04b-sklearn-classification-logis...,run-20230124200905,system.ExperimentRun,COMPLETE,bq://mg-ce-demos.fraud.fraud_prepped,04b,04_04b,gs://mg-ce-demos/04/04b/models/20230124200905/...,2.0,https://us-central1.tensorboard.googleusercont...,...,0.999087,0.607143,0.54902,0.878229,0.894737,0.774422,0.99921,0.809018,0.848485,0.618182
2,experiment-04-04b-sklearn-classification-logis...,run-20221116125513,system.ExperimentRun,COMPLETE,bq://mg-ce-demos.fraud.fraud_prepped,04b,04_04b,gs://mg-ce-demos/04/04b/models/20221116125513/...,2.0,https://us-central1.tensorboard.googleusercont...,...,0.999087,0.607143,0.54902,0.878229,0.894737,0.774422,0.99921,0.809018,0.848485,0.618182


Review the Experiments TensorBoard to compare runs:

In [48]:
print(f"The Experiment TensorBoard Link:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{exp.name}")

The Experiment TensorBoard Link:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+633472233130+locations+us-central1+tensorboards+4577495604850065408+experiments+experiment-04-04b-sklearn-classification-logistric-regression


In [49]:
expRun.get_time_series_data_frame()

""


### Compare This Run Using Experiments

Get a list of all experiments in this project:

In [50]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [51]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Combine the runs from all experiments in SERIES into a single dataframe:

In [52]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-04-04b-sklearn-classification-logistric-regression
experiment-04-04a-sklearn-classification-logistric-regression
experiment-04-04-sklearn-classification-logistic-regression
experiment-04-04c-sklearn-classification-logistric-regression
experiment-04-04-sklearn-classification-sklearn-logistic-regression
experiment-04-04-tf-classification-sklearn-logistic-regression


Create ranks for models within experiment and across the entire SERIES:

In [53]:
def ranker(metric = 'metric.test_roc_auc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_roc_auc')
ranks

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_roc_auc,series_rank,experiment_rank
59,experiment-04-04-sklearn-classification-logist...,run-20221102155936,04_04,2,0.803501,1.0,1.0
58,experiment-04-04-sklearn-classification-logist...,run-20221104083407,04_04,3,0.803501,1.0,1.0
57,experiment-04-04-sklearn-classification-logist...,run-20221104090606,04_04,4,0.803501,1.0,1.0
56,experiment-04-04-sklearn-classification-logist...,run-20221107104554,04_04,5,0.803501,1.0,1.0
55,experiment-04-04-sklearn-classification-logist...,run-20221108143538,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
64,experiment-04-04c-sklearn-classification-logis...,run-20221129083431,04_04c,5,0.803501,1.0,1.0
63,experiment-04-04c-sklearn-classification-logis...,run-20221129094800,04_04c,1,0.803501,1.0,1.0
62,experiment-04-04c-sklearn-classification-logis...,run-20221201132723,04_04c,2,0.803501,1.0,1.0
61,experiment-04-04c-sklearn-classification-logis...,run-20221206105306,04_04c,3,0.803501,1.0,1.0


In [54]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_roc_auc,series_rank,experiment_rank
0,experiment-04-04b-sklearn-classification-logis...,run-20230125091901,04_04b,3,0.803501,1.0,1.0


In [55]:
print(f"The current model is ranked {current_rank['experiment_rank'].iloc[0]} within this experiment and {current_rank['series_rank'].iloc[0]} across this series.")

The current model is ranked 1.0 within this experiment and 1.0 across this series.


### Create/Retrieve The Endpoint For This Series

In [56]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Endpoint Exists: projects/633472233130/locations/us-central1/endpoints/7186271659702091776
Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/7186271659702091776?project=mg-ce-demos


In [57]:
endpoint.display_name

'04'

In [58]:
endpoint.traffic_split

{}

In [59]:
deployed_models = endpoint.list_models()
#deployed_models

### Should This Model Be Deployed?
Is it better than the model already deployed on the endpoint?

In [60]:
deploy = False
if deployed_models:
    for deployed_model in deployed_models:
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'The current model/version is already deployed.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'The current model is ranked better ({model_rank}) than a currently deployed model ({deployed_rank}).')
            break
    if deploy == False: print(f'The current model is ranked worse ({model_rank}) than a currently deployed model ({deployed_rank})')
else: 
    deploy = True
    print('No models currently deployed.')

No models currently deployed.


### Deploy Model To Endpoint

In [61]:
if deploy:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Not deploying - current model is worse ({model_rank}) than the currently deployed model ({deployed_rank})')

Deploying model with 100% of traffic...
Deploying Model projects/633472233130/locations/us-central1/models/model_04_04b to Endpoint : projects/633472233130/locations/us-central1/endpoints/7186271659702091776
Deploy Endpoint model backing LRO: projects/633472233130/locations/us-central1/endpoints/7186271659702091776/operations/1550035211587158016
Endpoint model deployed. Resource name: projects/633472233130/locations/us-central1/endpoints/7186271659702091776


### Remove Deployed Models without Traffic

In [62]:
for deployed_model in endpoint.list_models():
    if deployed_model.id in endpoint.traffic_split:
        print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

Model 04_04b with version 3 has traffic = 100


In [63]:
endpoint.traffic_split

{'2786773793249427456': 100}

In [64]:
#endpoint.list_models()

---
## Prediction

See many more details on requesting predictions in the [05Tools - Prediction](./05Tools%20-%20Prediction.ipynb) notebook.

### Prepare a record for prediction: instance and parameters lists

In [65]:
test_query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE splits = 'TEST'"
test = bq.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(VAR_OMIT.split('-'))]
y_test = test[VAR_TARGET]

instances = [X_test.to_dict(orient='split')['data'][0]]

### Get Predictions: Python Client

In [66]:
prediction = endpoint.predict(instances=instances)
prediction

Prediction(predictions=[0.0], deployed_model_id='2786773793249427456', model_version_id='3', model_resource_name='projects/633472233130/locations/us-central1/models/model_04_04b', explanations=None)

In [67]:
prediction.predictions[0]

0.0

In [68]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [69]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": instances}))

In [70]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    0
  ],
  "deployedModelId": "2786773793249427456",
  "model": "projects/633472233130/locations/us-central1/models/model_04_04b",
  "modelDisplayName": "04_04b",
  "modelVersionId": "3"
}


### Get Predictions: gcloud (CLI)

In [71]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[0]


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



---
## Remove Resources
see notebook "99 - Cleanup"